<a href="https://colab.research.google.com/github/2203A51251/NLP_Lab/blob/main/Assignment8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Example text data (you can replace this with any larger corpus) text = """ Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her. [CO5]

(i) Build the Transformer Model on above dataset

(ii) Train the model using 20, 60, 70 epochs

 (iii) After training, use the model to generate new text by feeding it an initial seed text

(iv) Experimenting and Improving the Model by large dataset and hyper tune parameter.

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import numpy as np

In [3]:
text = """Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her."""

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
tokens = tokenizer.encode(text, return_tensors="pt")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))  # Adjust token embeddings for the tokenizer
model = model.to("cuda" if torch.cuda.is_available() else "cpu")


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
def train_transformer(model, tokens, epochs, learning_rate=1e-4):
    # Set up the optimizer
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    tokens = tokens.to(device)

    # Training loop
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()

        # Forward pass and compute loss
        outputs = model(tokens, labels=tokens)
        loss = outputs.loss
        loss.backward()
        # Update weights
        optimizer.step()

        # Print loss every few epochs
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f}")


In [7]:
# Training with 20 epochs
print("Training with 20 epochs:")
train_transformer(model, tokens, epochs=20)

# Training with 60 epochs
print("\nTraining with 60 epochs:")
train_transformer(model, tokens, epochs=60)

# Training with 70 epochs
print("\nTraining with 70 epochs:")
train_transformer(model, tokens, epochs=70)


Training with 20 epochs:
Epoch 10/20, Loss: 0.0486
Epoch 20/20, Loss: 0.0070

Training with 60 epochs:
Epoch 10/60, Loss: 0.0010
Epoch 20/60, Loss: 0.0012
Epoch 30/60, Loss: 0.1004
Epoch 40/60, Loss: 0.0003
Epoch 50/60, Loss: 0.0009
Epoch 60/60, Loss: 0.0001

Training with 70 epochs:
Epoch 10/70, Loss: 0.0000
Epoch 20/70, Loss: 0.0000
Epoch 30/70, Loss: 0.0000
Epoch 40/70, Loss: 0.0000
Epoch 50/70, Loss: 0.0003
Epoch 60/70, Loss: 0.0003
Epoch 70/70, Loss: 0.0055


In [8]:
def generate_text(model, tokenizer, seed_text, max_length=50):
    model.eval()
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Encode the seed text
    input_ids = tokenizer.encode(seed_text, return_tensors="pt").to(device)

    # Generate text
    with torch.no_grad():
        output = model.generate(input_ids, max_length=max_length, num_return_sequences=1, do_sample=True)

    # Decode the generated text
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Example generation
seed_text = "Once upon a time"
generated_text = generate_text(model, tokenizer, seed_text)
print("Generated text:", generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated text: Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the
